## Ablation study on KS dataset for PFNN latent dimension on short-term predictions

In [ ]:
cd ..

In [ ]:
import numpy as np
import pandas as pd
import torch
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

from model.utilities import *
from model.koopman_base import *
import sys
sys.path.append('./model')

import numpy.random as random

font = {'size'   : 12, 'family': 'Times New Roman'}
matplotlib.rc('font', **font)

In [ ]:
torch.manual_seed(0)
np.random.seed(0)

# Main
n_test = 100

sub = 4 # spatial subsample
S = 512
s = S//sub

T_in = 500 # skip first 100 seconds of each trajectory to let trajectory reach attractor
T = 200 # seconds to extract from each trajectory in data
T_out = T_in + T
step = 1 # Seconds to learn solution operator

# Load data
predloader = MatReader('../lake/data/KS.mat')
data_raw = predloader.read_field('u')
data_tensor = torch.tensor(data_raw, dtype=torch.float)[...,::sub]
data_test = data_tensor[-n_test:,:,:]

test_a = data_test[:,T_in-1:T_out-1,:].reshape(-1, s)
test_u = data_test[:,T_in:T_out,:].reshape(-1, s)
batch_size = 100
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(test_a, test_u), batch_size=batch_size, shuffle=False)

/var/folders/r1/v7355v5n0y3cw6l975b_dspm0000gn/T/ipykernel_15153/2715833728.py:19: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data_tensor = torch.tensor(data_raw, dtype=torch.float)[...,::sub]


In [ ]:
device = torch.device('cpu')

PFNN_latent_1_path = 'fill_PFNN_latent_128_model_path'
model_latent_1 = torch.load(PFNN_latent_1_path, map_location=device)

PFNN_latent_2_path = 'fill_PFNN_latent_256_model_path'
model_latent_2 = torch.load(PFNN_latent_2_path, map_location=device)

PFNN_latent_4_path = 'fill_PFNN_latent_512_model_path'
model_latent_4 = torch.load(PFNN_latent_4_path, map_location=device)

PFNN_latent_8_path = 'fill_PFNN_latent_1024_model_path'
model_latent_8 = torch.load(PFNN_latent_8_path, map_location=device)

In [ ]:
def episode_l2_loss(pred, truth, n = 100):
    return torch.mean((pred[:n] - truth[:n])**2)

def episode_loss_collection(regressive_steps, loss_fn, test_u, pred_1, pred_2, pred_3, pred_4):
      loss_dict = {}

      loss_latent_1 = loss_fn(pred_1, test_u, n=regressive_steps)
      loss_latent_2 = loss_fn(pred_2, test_u, n=regressive_steps)
      loss_latent_4 = loss_fn(pred_3, test_u, n=regressive_steps)
      loss_latent_8 = loss_fn(pred_4, test_u, n=regressive_steps)
            
      loss_dict['latent_1'] = loss_latent_1.item()
      loss_dict['latent_2'] = loss_latent_2.item()
      loss_dict['latent_4'] = loss_latent_4.item()
      loss_dict['latent_8'] = loss_latent_8.item()

      return loss_dict

In [ ]:
# collect L2 loss for each model
steps_n_list = np.array([100])
error_df_list = {}
for steps_n in steps_n_list:
      print('steps_n:', steps_n, 'started.')
      error_df = pd.DataFrame(columns=['latent_1', 'latent_2', 'latent_4', 'latent_8'])
      for init_id in tqdm(np.arange(n_test)):
            latent_1_long_pred = long_prediction(model_latent_1, test_a, init_id, 1, s, s, T=steps_n)
            latent_2_long_pred = long_prediction(model_latent_2, test_a, init_id, 1, s, s, T=steps_n)
            latent_4_long_pred = long_prediction(model_latent_4, test_a, init_id, 1, s, s, T=steps_n)
            latent_8_long_pred = long_prediction(model_latent_8, test_a, init_id, 1, s, s, T=steps_n)

            episode_loss_dict = episode_loss_collection(steps_n, episode_l2_loss, test_u[int(init_id*T):], latent_1_long_pred, latent_2_long_pred, latent_4_long_pred, latent_8_long_pred)
            error_df.loc[init_id] = episode_loss_dict
      error_df_list['step_{}'.format(steps_n)] = error_df

steps_n: 100 started.


100%|██████████| 100/100 [00:04<00:00, 21.13it/s]


In [ ]:
# get root mean squared error
error_mean_df = pd.DataFrame(
      columns=['latent_1', 'latent_2', 'latent_4', 'latent_8'])
for key in error_df_list.keys():
      error_mean_df.loc[key] = (np.sqrt(error_df_list[key])).mean()
error_std_df = pd.DataFrame(
      columns=['latent_1', 'latent_2', 'latent_4', 'latent_8'])
for key in error_df_list.keys():
      error_std_df.loc[key] = (np.sqrt(error_df_list[key])).std()

In [ ]:
error_mean_df

,latent_1,latent_2,latent_4,latent_8
step_100,1.381249,1.223732,0.989623,2.778672


In [ ]:
# find the range for normalization
range_list = []
for i in range (n_test):
      range_list.append((test_u[T*i:T*(i+1)].max() - test_u[T*i:T*(i+1)].min()).item())
range_list = np.array(range_list)
range_list_rep = range_list[:,None].repeat(6, axis=1)
range_mean = range_list.mean()
range_max = range_list.max()
print('range_mean:', range_mean, 'range_max:', range_max)

range_mean: 6.3963081169128415 range_max: 6.994479179382324


In [ ]:
error_mean_percent_df = 100*error_mean_df/range_mean.item()
error_std_percent_df = 100*error_std_df/range_mean.item()

### NRMSE in percentage (for 100 steps prediction) ablation results for model trained on different latent dimensions Latent_x (Dimension = x*128)

In [ ]:
error_mean_percent_df

,latent_1,latent_2,latent_4,latent_8
step_100,21.594466,19.131848,15.471778,43.441812


In [ ]:
error_std_percent_df

,latent_1,latent_2,latent_4,latent_8
step_100,1.49038,1.725468,1.995285,7.228813
